In [1]:
import logging

import openai

In [7]:
# Put anything you want in `API key`
openai.api_key = 'FreeTheModels'

# Point to your own url
openai.api_base = "http://localhost:8080/openai"
print(openai.Model.list())

{
  "data": [
    {
      "created": 0,
      "id": "all-minilm-l6-v2",
      "object": "",
      "owned_by": "",
      "permission": [],
      "root": "",
      "parent": ""
    },
    {
      "created": 0,
      "id": "ctransformers",
      "object": "",
      "owned_by": "Defense Unicorns",
      "permission": [],
      "root": "",
      "parent": ""
    },
    {
      "created": 0,
      "id": "text-embedding-ada-002",
      "object": "",
      "owned_by": "Defense Unicorns",
      "permission": [],
      "root": "",
      "parent": ""
    },
    {
      "created": 0,
      "id": "whisper-1",
      "object": "",
      "owned_by": "Defense Unicorns",
      "permission": [],
      "root": "",
      "parent": ""
    },
    {
      "created": 0,
      "id": "mpt-7b-chat",
      "object": "",
      "owned_by": "",
      "permission": [],
      "root": "",
      "parent": ""
    }
  ]
}


## `/completion` Endpoint

In [3]:
messages = [
    {"role": "system", "content": "You are a helpful programming assistant."},
    {"role": "user", "content": "Can you write a python program to download a series of files from s3 and search the contents of each file for email addresses?"},
]

In [4]:
# Prompt Templates
SYSTEM_FORMAT = "<|im_start|>system\n{}<|im_end|>\n"
USER_FORMAT = "<|im_start|>user\n{}<|im_end|>\n"
ASSISTANT_FORMAT = "<|im_start|>assistant\n{}<|im_end|>\n"
# what gets appended to the end of the prompt to open the assistant's part of the conversation
RESPONSE_PREFIX = ASSISTANT_FORMAT.split("{}")[0]

def chat_items_to_prompt(chat_items: list[dict]) -> str:
    """Converts a repeated ChatItem from a ChatCompletionRequest proto into a string

    This is the actual string that gets fed into the model to generate the outputs
    """
    prompt = ""
    for item in chat_items:
        if item["role"] == "system":
            prompt += SYSTEM_FORMAT.format(item["content"])
        elif item["role"] == "assistant":
            prompt += ASSISTANT_FORMAT.format(item["content"])
        elif item["role"] == "user":
            prompt += USER_FORMAT.format(item["content"])
        else:
            logging.warning(
                f"ChatRole {item['role']} is not implemented for this model and this ChatItem will be ignored."
            )
    # add the response prefix to start the model's reponse
    prompt += RESPONSE_PREFIX
    return prompt

In [5]:
prompt = chat_items_to_prompt(messages)

### Non-streaming case

In [6]:
response = openai.Completion.create(
    model="mpt-7b-chat",  # Select the appropriate engine for your needs
    prompt=prompt,
)

response

<OpenAIObject  id=903c26c0-92ba-45cb-bdab-37fb37ab644a at 0x7f2ad0107dd0> JSON: {
  "id": "903c26c0-92ba-45cb-bdab-37fb37ab644a",
  "object": "",
  "created": 1688959355,
  "model": "mpt-7b-chat",
  "choices": [
    {
      "text": "Sure, here's a sample Python program that downloads a series of files from S3 and searches their contents for email addresses:\n```python\nimport boto3\nimport re\n# Set up S3 client\ns3 = boto3.client('s3')\n# Define bucket and prefix\nbucket_name = 'your-bucket-name'\nprefix = 'your-prefix'\n# Download files from S3\nfiles = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']\n# Loop through files and search for email addresses\nfor file in files:\n    file_key = file['Key']\n    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()\n    email_regex = r'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b'\n    emails = re.findall(email_regex, file_obj)\n    if emails:\n        print(f\"Found email addresses in {file_ke

In [7]:
print(response["choices"][0]["text"])

Sure, here's a sample Python program that downloads a series of files from S3 and searches their contents for email addresses:
```python
import boto3
import re
# Set up S3 client
s3 = boto3.client('s3')
# Define bucket and prefix
bucket_name = 'your-bucket-name'
prefix = 'your-prefix'
# Download files from S3
files = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']
# Loop through files and search for email addresses
for file in files:
    file_key = file['Key']
    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(email_regex, file_obj)
    if emails:
        print(f"Found email addresses in {file_key}: {emails}")
```
This program uses the `boto3` library to interact with S3. It first sets up an S3 client and defines the bucket name and prefix (i.e. the directory where the files are located). It then downloads all the files from S3 using the `list_ob

### Streaming Case

In [8]:
response = openai.Completion.create(
    model="mpt-7b-chat",  # Select the appropriate engine for your needs
    prompt=prompt,
    stream=True  # this time, we set stream=True
)

for chunk in response:
    print(chunk["choices"][0]["text"], end="")

Yes, I can help you with that. Here's a sample Python program that uses the boto3 library to download files from S3 and searches their contents for email addresses:
```python
import boto3
import re
# Set up S3 client
s3 = boto3.client('s3')
# Define bucket and key
bucket_name = 'your-bucket-name'
object_key = 'path/to/object'
# Download file from S3
response = s3.get_object(Bucket=bucket_name, Key=object_key)
file_content = response['Body'].read().decode('utf-8')
# Search for email addresses in file content
email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
emails = re.findall(email_regex, file_content)
# Print found emails
if emails:
    print('Email addresses found:')
    for email in emails:
        print(email)
else:
    print('No email addresses found')
```
This program assumes that you have already set up an S3 bucket and uploaded the files you want to download. You will also need to have the boto3 library installed.
The program downloads the file from S3, searc

## `/chat/completions` Endpoint

### Non-streaming Case

In [9]:
response = openai.ChatCompletion.create(
    model='mpt-7b-chat',
    messages=messages,
)

response

<OpenAIObject  id=4c41f73c-a5da-49cf-b302-7cacbb8e64ae at 0x7f2ad16ea5c0> JSON: {
  "id": "4c41f73c-a5da-49cf-b302-7cacbb8e64ae",
  "object": "",
  "created": 1688959422,
  "model": "mpt-7b-chat",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Sure, here's a Python program that downloads a series of files from S3 and searches their contents for email addresses:\n```python\nimport boto3\nimport re\n# Set up S3 client\ns3 = boto3.client('s3')\n# Define bucket and prefix\nbucket_name = 'your-bucket-name'\nprefix = 'your-prefix'\n# Download files from S3\nfiles = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']\n# Search contents of each file for email addresses\nfor file in files:\n    file_key = file['Key']\n    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()\n    email_regex = r'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b'\n    emails = re.findall(email_regex, file_obj)\n   

In [10]:
print(response["choices"][0]["message"]["content"])

Sure, here's a Python program that downloads a series of files from S3 and searches their contents for email addresses:
```python
import boto3
import re
# Set up S3 client
s3 = boto3.client('s3')
# Define bucket and prefix
bucket_name = 'your-bucket-name'
prefix = 'your-prefix'
# Download files from S3
files = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']
# Search contents of each file for email addresses
for file in files:
    file_key = file['Key']
    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(email_regex, file_obj)
    print(f"File '{file_key}' contains the following email addresses:")
    for email in emails:
        print(email)
```
This program uses the `boto3` library to interact with S3 and download files. It then searches the contents of each file for email addresses using regular expressions. The regular expression used here is `

### Streaming Case

In [4]:
response = openai.ChatCompletion.create(
    model='mpt-7b-chat',
    messages=messages,
    stream=True  # this time, we set stream=True
)

for chunk in response:
    print(chunk["choices"][0]["message"]["content"], end="")

Yes, I can help you with that. Here's a sample Python program that downloads a series of files from S3 and searches their contents for email addresses:
```python
import boto3
import re
# Set up S3 client
s3 = boto3.client('s3')
# Define bucket and prefix
bucket_name = 'your-bucket-name'
prefix = 'your-prefix'
# Download files from S3
files = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']
# Loop through files and search for email addresses
for file in files:
    file_key = file['Key']
    file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(email_regex, file_obj)
    print(f"File {file_key}:")
    print(emails)
```
This program uses the `boto3` library to interact with S3. It first sets up an S3 client and defines the bucket name and prefix (i.e. the directory where the files are located). It then downloads all the files from S3 using the `list_objects

KeyboardInterrupt: 

## Embeddings

In [14]:
response = openai.Embedding.create(
    model = "all-minilm-l6-v2",
    input = "This is a super cool text embedding"
)

response

<OpenAIObject  at 0x7f95f52c71a0> JSON: {
  "object": "",
  "data": [
    {
      "object": "",
      "embedding": [
        -0.042698976,
        -0.026150815,
        0.061529085,
        -0.017067522,
        0.039560657,
        0.02837218,
        0.031743776,
        -0.0143780075,
        0.08584679,
        -0.001684193,
        0.044729978,
        0.1055661,
        0.049236063,
        0.013457378,
        -0.11796918,
        0.059610043,
        0.10469074,
        0.02386002,
        -0.081601895,
        -0.05700545,
        -0.056711912,
        -0.009005215,
        0.084444866,
        -0.0627069,
        0.022999374,
        0.04371579,
        -0.08111302,
        0.07167676,
        0.061417066,
        0.0068834806,
        0.0067532244,
        -0.0013206801,
        0.012589719,
        0.008909394,
        -0.0050711646,
        0.031988457,
        0.031716038,
        0.0700321,
        -0.023534963,
        0.04725706,
        0.003399456,
        -0.0570747